In [2]:
import pandas as pd
import numpy as np

In [3]:
substring = "Exch0"
# read the single file
LOB = []
file = "C:/Users/14620/Desktop/test.txt"
file1 = "E:/mini project bristol/JPMorgan_Set01/JPMorgan_Set01/LOBs/UoB_Set01_2025-01-02LOBs.txt"

with open(file1, 'r') as f:
    for line in f:
        new_line = line.replace(substring, f'"{substring}"')  #First, because Exch0 is a string, the string is first wrapped in quotes to prevent eval () from mistaking Exch0 for a variable name
        LOB.append(eval(new_line.strip('\n')))
LOB[:5]

[[0.0, 'Exch0', [['bid', []], ['ask', []]]],
 [0.279, 'Exch0', [['bid', [[1, 6]]], ['ask', []]]],
 [1.333, 'Exch0', [['bid', [[1, 6]]], ['ask', [[800, 1]]]]],
 [1.581, 'Exch0', [['bid', [[1, 6]]], ['ask', [[799, 1]]]]],
 [1.643, 'Exch0', [['bid', [[1, 6]]], ['ask', [[798, 1]]]]]]

In [6]:
def dictionary(a):
    it = iter(a)
    res_dct = dict(zip(it, it))
    return res_dct


#Method: Use iterators to convert them into dictionary form

New_data = []
test_lines = np.asarray(LOB, dtype=object)  #The LOB is converted to a multidimensional array using the asarray method
test_lines[:5]

array([[0.0, 'Exch0', list([['bid', []], ['ask', []]])],
       [0.279, 'Exch0', list([['bid', [[1, 6]]], ['ask', []]])],
       [1.333, 'Exch0', list([['bid', [[1, 6]]], ['ask', [[800, 1]]]])],
       [1.581, 'Exch0', list([['bid', [[1, 6]]], ['ask', [[799, 1]]]])],
       [1.643, 'Exch0', list([['bid', [[1, 6]]], ['ask', [[798, 1]]]])]],
      dtype=object)

In [7]:
for i in test_lines:
    every_line = [["Time", i[0]], ["Encode", i[1]], i[2][0], i[2][1]]
    #Create a new column and put the data in the testlines array into the new format for easy computation below
    New_data.append(every_line)


for i in New_data:  #Put the data into the convert method and convert it all into dictionary format
    i[0] = dictionary(i[0])
    i[1] = dictionary(i[1])
    i[2] = dictionary(i[2])
    i[3] = dictionary(i[3])
    i[0] = i[0] | i[1] | i[2] | i[3]  #Summarize all columns into the first column
New_data = np.delete(New_data, [1, 2, 3], 1)  #Delete redundant columns
New_data[:5]

array([[{'Time': 0.0, 'Encode': 'Exch0', 'bid': [], 'ask': []}],
       [{'Time': 0.279, 'Encode': 'Exch0', 'bid': [[1, 6]], 'ask': []}],
       [{'Time': 1.333, 'Encode': 'Exch0', 'bid': [[1, 6]], 'ask': [[800, 1]]}],
       [{'Time': 1.581, 'Encode': 'Exch0', 'bid': [[1, 6]], 'ask': [[799, 1]]}],
       [{'Time': 1.643, 'Encode': 'Exch0', 'bid': [[1, 6]], 'ask': [[798, 1]]}]],
      dtype=object)

In [9]:
New_data2 = New_data.tolist()
df_test = pd.DataFrame(New_data2)
df_test[:5]

,0
0,"{'Time': 0.0, 'Encode': 'Exch0', 'bid': [], 'a..."
1,"{'Time': 0.279, 'Encode': 'Exch0', 'bid': [[1,..."
2,"{'Time': 1.333, 'Encode': 'Exch0', 'bid': [[1,..."
3,"{'Time': 1.581, 'Encode': 'Exch0', 'bid': [[1,..."
4,"{'Time': 1.643, 'Encode': 'Exch0', 'bid': [[1,..."


In [12]:
data_test = New_data.flatten()
data_test[:5]

array([{'Time': 0.0, 'Encode': 'Exch0', 'bid': [], 'ask': []},
       {'Time': 0.279, 'Encode': 'Exch0', 'bid': [[1, 6]], 'ask': []},
       {'Time': 1.333, 'Encode': 'Exch0', 'bid': [[1, 6]], 'ask': [[800, 1]]},
       {'Time': 1.581, 'Encode': 'Exch0', 'bid': [[1, 6]], 'ask': [[799, 1]]},
       {'Time': 1.643, 'Encode': 'Exch0', 'bid': [[1, 6]], 'ask': [[798, 1]]}],
      dtype=object)

In [13]:
data_test2 = data_test.tolist()  #Before converting a dict directly to a dataframe, it must be converted to a list
data_test2
df = pd.DataFrame(data_test2)
df = df.set_index("Time")
df.head(5)

,Encode,bid,ask
Time,,,
0.000,Exch0,[],[]
0.279,Exch0,"[[1, 6]]",[]
1.333,Exch0,"[[1, 6]]","[[800, 1]]"
1.581,Exch0,"[[1, 6]]","[[799, 1]]"
1.643,Exch0,"[[1, 6]]","[[798, 1]]"


In [14]:
df1 = df.copy()

In [15]:
def LOBs_df(df):
    best_bid_price = []
    best_ask_price = []
    bid_volume = []
    ask_volume = []
    bid_ask_spread = []
    median_price = []

    #important variable
    #in some range of price, The cumulative number of bid and ask orders called book depth
    book_Depth = []
    #The difference in quantity between a bid order and a ask order at a given price level or in the entire order book. This can be used to gauge the bias of market participants' sentiment, which can affect short-term price movements.
    order_Imbalance = []

    for bid_line, ask_line in zip(df['bid'], df['ask']):  #iterrows function, calculate those things in each row
        if len(bid_line) > 0:  #when bid_line >0,which means we have bid price
            best_bid_price.append(bid_line[0][0])  #add first bid price
            bid_volume.append(bid_line[0][1])  #add first bid volume
        else:  # which means we do not have bid price(NaN)
            best_bid_price.append(np.nan)
            bid_volume.append(np.nan)

        if len(ask_line) > 0:  #same as above
            best_ask_price.append(ask_line[0][0])
            ask_volume.append(ask_line[0][1])
        else:  # which means we do not have ask price(NaN)
            best_ask_price.append(np.nan)
            ask_volume.append(np.nan)

        if len(bid_line) > 0 and len(ask_line) > 0:  #we have all the bid and ask price
            bid_ask_spread.append(ask_line[0][0] - bid_line[0][0])
            median_price.append((ask_line[0][0] + bid_line[0][0]) / 2)
        else:
            bid_ask_spread.append(np.nan)
            median_price.append(np.nan)

        bid_depth = bid_line[0][1] if len(bid_line) > 0 else 0
        ask_depth = ask_line[0][1] if len(ask_line) > 0 else 0
        book_Depth.append(bid_depth + ask_depth)
        order_Imbalance.append(bid_depth - ask_depth)


    df.insert(1, "bid_ask_spread", bid_ask_spread)
    df.insert(2, "Mid_Price", median_price)
    df.insert(3, "Best_bid_price", best_bid_price)
    df.insert(4, "bid_volume", bid_volume)
    df.insert(5, "Best_ask_price", best_ask_price)
    df.insert(6, "ask_volume", ask_volume)
    df.insert(7, "book_Depth", book_Depth)
    df.insert(8, "order_Imbalance", order_Imbalance)

    return df

In [18]:
df2=LOBs_df(df1)
df2=df2.drop(columns=["bid", "ask"])

ValueError: cannot insert bid_ask_spread, already exists

In [20]:
df2[:5]

,Encode,bid_ask_spread,Mid_Price,Best_bid_price,bid_volume,Best_ask_price,ask_volume,book_Depth,order_Imbalance
Time,,,,,,,,,
0.000,Exch0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
0.279,Exch0,NaN,NaN,1.0,6.0,NaN,NaN,6,6
1.333,Exch0,799.0,400.5,1.0,6.0,800.0,1.0,7,5
1.581,Exch0,798.0,400.0,1.0,6.0,799.0,1.0,7,5
1.643,Exch0,797.0,399.5,1.0,6.0,798.0,1.0,7,5
1.736,Exch0,537.0,529.5,261.0,1.0,798.0,1.0,2,0
1.984,Exch0,536.0,529.0,261.0,1.0,797.0,1.0,2,0
2.015,Exch0,77.0,299.5,261.0,1.0,338.0,3.0,4,-2
2.139,Exch0,36.0,279.0,261.0,1.0,297.0,5.0,6,-4


In [22]:
df2.to_csv("E:/mini project bristol/processed data(LOBS,tapes)/processed_LOBs.csv")